#### 交互式了解Aestas的使用方法

逐条执行下面的代码块，了解Aestas的使用方法。
在开始之前，确保.NET Interactive，pwsh已经安装

pwsh可以用包管理器安装。在Windows上，请考虑使用
```
winget install Microsoft.PowerShell
```

In [1]:
$aestasdoc = Get-Location
$aestas = Split-Path -Path $aestasdoc
$aestasdll = (Convert-Path ../bin/Release/net8.0/aestas.dll).Replace("\", "/")

首先编译Aestas

In [ ]:
cd $aestas
./aestas.ps1 build

fsi并不支持相对路径，所以把获得的绝对路径保存到临时脚本中，让路径和这个notebook文件无关的同时还能加载Aestas

In [2]:
cd $aestasdoc
"#r `"$aestasdll`"" | Out-File .load.fsx

In [1]:
#r "nuget:FSharpPlus"
#load ".load.fsx"
open FSharpPlus
open Aestas
open Aestas.Prim
open Aestas.Core
open Aestas.Core.Builtin
open Aestas.Core.AestasBot
open Aestas.Core.CommandExecuter

Installed Packages FSharpPlus, 1.6.1

创建Domain

*VirtualDomain*是一个虚拟的Domain，我们借助它的接口和它绑定的Bot来交互

In [2]:
let botName = "Vespera"
let chatHistory = arrList<string*(AestasContent list)>()
let virtualDomain = VirtualDomain((fun mid contents -> chatHistory.Add(botName, contents)),
    ignore, {
        uid = 1u
        name = botName
    },{
        uid = 0u
        name = "You"
    }, 0u, "Test", true)
let sendToBot x = 
    let contents = [AestasText x]
    chatHistory.Add("You", contents)
    match virtualDomain.Input contents |> Async.RunSynchronously with
    | Ok (), _ -> chatHistory[max (chatHistory.Count-2) 0..].DisplayTable() |> ignore
    | Error msg, _ -> printfn "Error: %s" msg

编写一个最简单的“语言模型”代码，让Bot能够重复它的输入

In [3]:
type Repeater() =
    interface ILanguageModelClient with
        member _.GetReply bot domain = 
            async {
                return chatHistory[^0] |> snd |> Ok, ignore
            }
        member _.CacheMessage bot domain message = ()
        member _.CacheContents bot domain contents = ()
        member _.ClearCache domain = ()
        member _.RemoveCache domain messageID = ()

创建一个Bot. 这次用的函数相比之前的教程更复杂，之后会解释参数的含义

In [4]:
let bot = createBot {|
    name = botName
    model = Repeater()
    systemInstruction = Some ""
    systemInstructionBuilder = None
    contentLoadStrategy = None
    contentParseStrategy = None
    messageReplyStrategy = Some StrategyReplyAll
    messageCacheStrategy = Some StrategyCacheAll
    contextStrategy = None
    inputPrefixBuilder = None
    userCommandPrivilege = None
|}
bindDomain bot virtualDomain

在之后可以执行下面的单元格清除所有上下文，以便重新开始

In [5]:
bot.ClearCachedContext virtualDomain
chatHistory.Clear()

在之后可以执行下面的单元格来显示上下文

In [6]:
chatHistory.DisplayTable()

尝试发送第一条消息

In [7]:
sendToBot "Hello"

Item1 Item2 You [ AestasText "Hello" ] HeadOrDefault AestasText "Hello" Item Hello TailOrNull [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: 输入列表为空。\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method169(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T i... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663764 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\User\.nuget\packages\microsoft.dotnet-interactive\1.0.522904\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140731782786272 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] ReturnParameter T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes (empty) Message 输入列表为空。 Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method169(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: 输入列表为空。\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\r\n at lambda_method170(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T i... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663765 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\User\.nuget\packages\microsoft.dotnet-interactive\1.0.522904\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140731782786288 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList<T> ReturnTypeCustomAttributes Microsoft.FSharp.Collection

为Bot添加指令系统

In [8]:
addCommandExecuter bot "/" (makeExecuterWithBuiltinCommands [])
bot.CommandExecuters

key value / Aestas.Core+Builtin+SpacedTextCommandExecuter Commands index value 0 { name = "version"\n description = "Print the version of Aestas"\n accessibleDomain = All\n privilege = Normal\n execute = <fun:versionCommand@869> } name version description Print the version of Aestas accessibleDomain All privilege Normal execute Aestas.Core+Builtin+versionCommand@869 1 { name = "clear"\n description = "Clear the cached context"\n accessibleDomain = All\n privilege = Normal\n execute = <fun:clearCommand@877> } name clear description Clear the cached context accessibleDomain All privilege Normal execute Aestas.Core+Builtin+clearCommand@877 2 { name = "help"\n description = "List all commands"\n accessibleDomain = All\n privilege = Normal\n execute = <fun:helpCommand@886> } name help description List all commands accessibleDomain All privilege Normal execute Aestas.Core+Builtin+helpCommand@886 3 { name = "echo"\n description = "Repeat the input"\n accessibleDomain = All\n privilege = Normal\n execute = <fun:echoCommand@898> } name echo description Repeat the input accessibleDomain All privilege Normal execute Aestas.Core+Builtin+echoCommand@898

尝试一条指令

In [9]:
sendToBot "/version"

Item1 Item2 You [ AestasText "/version" ] HeadOrDefault AestasText "/version" Item /version TailOrNull [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: 输入列表为空。\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method169(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T i... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663764 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\User\.nuget\packages\microsoft.dotnet-interactive\1.0.522904\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140731782786272 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] ReturnParameter T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes (empty) Message 输入列表为空。 Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method169(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: 输入列表为空。\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\r\n at lambda_method170(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T i... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663765 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\User\.nuget\packages\microsoft.dotnet-interactive\1.0.522904\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140731782786288 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList<T> ReturnTypeCustomAttributes Microsoft.FSharp.C

编写命令*add2*，把输入的两个数字相加并发送出来

In [10]:
tryAddCommand bot.CommandExecuters["/"] {
    name = "add2"
    description = "Add two numbers"
    accessibleDomain = CommandAccessibleDomain.All
    privilege = CommandPrivilege.Normal
    execute = fun executer env args ->
        match args with
        | [|str0; str1|] ->
            let i0, i1 = trySscanf "%d" str0, trySscanf "%d" str1
            match i0, i1 with
            | Some i0, Some i1 ->
                i0 + i1 |> string |> env.log
            | _ ->  env.log "Invalid arguments"
        | _ -> env.log "Invalid arguments"
} |> printfn "%A"

Ok ()


尝试刚才编写的指令

In [11]:
sendToBot "/add2 2 3"
sendToBot "/add2 2"

Item1 Item2 You [ AestasText "/add2 2 3" ] HeadOrDefault AestasText "/add2 2 3" Item /add2 2 3 TailOrNull [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: 输入列表为空。\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method169(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T i... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663764 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\User\.nuget\packages\microsoft.dotnet-interactive\1.0.522904\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140731782786272 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] ReturnParameter T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes (empty) Message 输入列表为空。 Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method169(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: 输入列表为空。\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\r\n at lambda_method170(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T i... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663765 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\User\.nuget\packages\microsoft.dotnet-interactive\1.0.522904\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140731782786288 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList<T> ReturnTypeCustomAttributes Microsoft.FShar

Item1 Item2 You [ AestasText "/add2 2" ] HeadOrDefault AestasText "/add2 2" Item /add2 2 TailOrNull [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: 输入列表为空。\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method169(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T i... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663764 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\User\.nuget\packages\microsoft.dotnet-interactive\1.0.522904\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140731782786272 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] ReturnParameter T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes (empty) Message 输入列表为空。 Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method169(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: 输入列表为空。\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\r\n at lambda_method170(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T i... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663765 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\User\.nuget\packages\microsoft.dotnet-interactive\1.0.522904\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140731782786288 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList<T> ReturnTypeCustomAttributes Microsoft.FSharp.Coll

了解ContentParser. 首先，写一个会使用ContentParser的“语言模型”

In [14]:
type AlwaysBlank() =
    interface ILanguageModelClient with
        member _.GetReply bot domain =
            async {
                return "#[blank]" |> modelOutputParser bot domain |> Ok, ignore
            }
        member _.CacheMessage bot domain message = ()
        member _.CacheContents bot domain contents = ()
        member _.ClearCache domain = ()
        member _.RemoveCache domain messageID = ()

执行下面的代码，观察输出

In [15]:
AlwaysBlank() |> bindModel bot
sendToBot "Hello"

Item1 Item2 You [ AestasText "Hello" ] HeadOrDefault AestasText "Hello" Item Hello TailOrNull [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: 输入列表为空。\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method169(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T i... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663764 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\User\.nuget\packages\microsoft.dotnet-interactive\1.0.522904\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140731782786272 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] ReturnParameter T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes (empty) Message 输入列表为空。 Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method169(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: 输入列表为空。\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\r\n at lambda_method170(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T i... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663765 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\User\.nuget\packages\microsoft.dotnet-interactive\1.0.522904\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140731782786288 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList<T> ReturnTypeCustomAttributes Microsoft.FSharp.Collection